In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp37-none-any.whl size=45216 sha256=aa1c00dc941a8a2e71759a1810c8ca01c57946867cbe96aeb568060dd13164b6
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/', force_remount = True)
os.chdir('/content/drive/My Drive/Speech Recognition Model')

Mounted at /content/drive/


In [ ]:
import anvil.server

anvil.server.connect("IJ47VSBBPUWQH3HRT6LC2YHK-4U3N3PTKPEUCAXA3")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [ ]:
import time
import librosa
import soundfile
import os, glob, pickle
import numpy as np

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
  with soundfile.SoundFile(file_name) as sound_file:
    X = sound_file.read(dtype="float32")
    sample_rate=sound_file.samplerate
    if chroma:
      stft=np.abs(librosa.stft(X))
      result=np.array([])
    if mfcc:
      mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
      result=np.hstack((result, mfccs))
    if chroma:
      chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
      result=np.hstack((result, chroma))
    if mel:
      mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
      result=np.hstack((result, mel))
  return result

In [ ]:
import anvil.media
from pickle import load

model = load(open('speech_emotion_model.pkl', 'rb'))
 
@anvil.server.callable
def predict_voice(file):
  with anvil.media.TempFile(file) as f:
    try:
      feature = extract_feature(f, mfcc = True, chroma = True, mel = True)
      feature = feature.reshape(-1, 1).T
      score = model.predict(feature)
      return "The emotion of audio is "+score[0]
    except:
      return "Some error in uploaded file"


In [ ]:
anvil.server.wait_forever()

KeyboardInterrupt: ignored